In [1]:
1+1

2

In [27]:
using SparseArrays
using LinearAlgebra
using Arpack
using Statistics
using Random
using DelimitedFiles
using NPZ
using ExpmV
using Dates

Random.seed!(Dates.now().instant.periods.value)

TaskLocalRNG()

In [ ]:
function get_potts_operators()
    id = sparse(ComplexF64[1 0 0; 0 1 0; 0 0 1])
    Z = sparse(ComplexF64[1 0 0; 0 exp(2im * pi / 3) 0; 0 0 exp(4im * pi / 3)]) ## τ in Romain's paper
    X = sparse(ComplexF64[0 1 0; 0 0 1; 1 0 0]) ## σ in Romain's paper

    return id, Z, X
end

function build_term(operators::Vector{<:SparseMatrixCSC}) #
    term = operators[1]
    for j in 2:length(operators)
        term = kron(term, operators[j])
    end
    return term
end

function create_local_Z_operator(L::Int, site::Int)
    _, Z, _ = get_potts_operators()
    ops = fill(id, L)
    ops[site] = Z
    return build_term(ops)
end

function create_local_Z2_operator(L::Int, site::Int)
    _, Z, _ = get_potts_operators()
    ops = fill(id, L)
    ops[site] = Z * Z
    return build_term(ops)
end

function total_Sz2(L::Int)
    dim = 3^L
    Sz2_tot = spzeros(ComplexF64, dim, dim)
    for i in 1:L
        Sz2_tot += create_local_sz2_operator(L, i)
    end
    return Sz2_tot
end

total_Sz2 (generic function with 1 method)

In [29]:
function Hamiltonian(L::Int)
    id, Z, X = get_potts_operators()
    dim = 3^L
    
    # Initialize Hamiltonian; J = 1
    H = spzeros(ComplexF64, dim, dim)
    
    for i in 1:L
        # Periodic boundary condition
        ip = mod1(i + 1, L)

        J = rand()
        g = rand()
        
        ops_H_1 = fill(id, L)
        ops_H_1[i] = X'
        ops_H_1[ip] = X
        H += J * build_term(ops_H_1)
        
       
        ops_H_2 = fill(id, L)
        ops_H_2[i] = Z
        H += g * build_term(ops_H_2)
    end
    
    H += adjoint(H)

    return H
end

Hamiltonian (generic function with 1 method)

In [30]:
Hamiltonian(4)

81×81 SparseMatrixCSC{ComplexF64, Int64} with 729 stored entries:
⎡⢑⢔⠌⠌⣂⠀⠀⠈⠢⠀⠈⠢⡀⠐⠌⠀⡀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠠⠑⢀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⎤
⎢⡂⠅⢕⢕⠀⠑⢄⠀⠀⢄⠀⠀⠈⠂⠀⠑⢀⠠⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠂⠁⢄⠀⠀⠑⢄⠀⠀⠀⠀⠀⎥
⎢⠈⠘⢄⠀⠑⢄⢄⢃⠢⠀⠁⠀⠢⡐⢄⠀⠀⠁⢄⠂⠀⠀⠀⠀⠀⠀⠁⠀⠀⠈⠀⡐⠄⠀⠀⠑⢄⠀⠀⠀⎥
⎢⡀⠀⠀⠑⠤⢑⢑⢔⠌⠑⢄⠀⠀⠀⠀⠑⢄⠀⠀⠰⠁⢀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⠑⢀⠀⠀⠑⢄⠀⎥
⎢⠈⠂⠀⢄⠈⠂⢆⠁⠑⢄⢀⠱⠠⡀⠀⠀⠀⠑⢄⠀⠈⠂⡀⠄⠀⠀⠠⡀⠀⠀⠀⠀⠀⠀⠂⠁⢄⠀⠀⠑⎥
⎢⠢⡀⠀⠀⠁⠀⠀⠑⢄⡐⠕⢅⢅⠂⠀⠀⠀⠀⠀⠱⡀⠀⠈⠠⡐⠀⠀⠈⠢⡀⠀⠀⠀⠀⠀⠈⠀⡐⠄⠀⎥
⎢⢀⠈⠢⠀⢈⠢⠀⠀⠀⠢⠡⠑⠑⢄⠀⠀⡀⡀⠀⠀⢈⠢⠀⢀⠀⠢⠁⢀⠀⠈⠢⠀⠀⠀⠀⢀⠀⠀⠠⠑⎥
⎢⠂⠁⢄⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⢕⢕⢐⠨⢄⠀⠀⠑⠀⠀⠑⢄⠈⠂⡀⠄⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⎥
⎢⠀⠈⠀⡐⠄⠀⠀⠑⢄⠀⠀⠀⠀⠨⡐⡐⠕⢅⠀⠱⡀⠀⠢⡀⠀⠀⠁⠀⠈⠠⡐⠀⠀⠀⠀⠀⠀⠀⠑⢄⎥
⎢⠀⠀⠀⠀⠠⠑⢀⡀⠀⠑⢄⡀⠀⠀⠀⠑⢄⡀⢑⣴⢡⢑⣀⠀⠀⠑⠌⠢⣀⠀⠀⠢⢁⢀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠁⢀⠢⠀⠀⠈⠢⡐⢄⠀⠀⠈⢅⢒⠕⢅⠀⠑⢄⠀⠀⠀⠀⠑⢄⠀⠀⠁⢄⠂⠀⠀⠀⠀⎥
⎢⠑⢄⠀⠀⠀⠀⠀⠀⠀⠌⠂⡀⠀⢀⠀⠀⠈⠢⠀⠘⢄⠀⢑⢔⠌⠌⡂⠀⠀⠀⠀⠑⢄⠀⠀⠐⠌⠀⡀⠀⎥
⎢⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠐⠈⠠⡀⠑⢄⠀⠀⢄⠀⠀⠑⡂⠅⢕⢕⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠑⢀⠠⎥
⎢⢄⠂⠀⠀⠁⠀⠀⠀⠀⠢⡀⠀⠁⢀⠢⠀⠁⠀⠢⡁⠀⠀⠈⠈⠀⠀⠑⢄⢄⢂⠢⠀⠀⠀⠢⡁⠀⠢⡀⠁⎥
⎢⠀⠐⠌⠀⡀⠀⠀⠀⠀⠀⠈⠢⡀⠀⠀⠌⠂⡀⠀⠘⢄⠀⠀⠀⠀⠀⠠⢑⢑⢔⠌⠑⢄⠀⠀⢀⠀⠀⠈⠢⎥
⎢⢄⠀⠀⠑⢀⠠⠀⠀⠀⠀⠀⠀⠈⠂⠀⠀⠐⠈⠠⡀⠀⠑⢄⠀⠀⠀⠈⠂⢆⠁⠑⢄⢀⠱⠠⡀⠑⠀⠠⡀⎥
⎢⠀⠑⢄⠀⠀⠁⢄⠂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠁⢐⠄⠀⠀⠑⢄⠀⠀⠀⠀⠑⢄⡐⠕⢅⢅⠒⢄⠀⠀⠈⎥
⎢⠀⠀⠀⠑⢄⠀⠀⠐⠌⠀⡀⠀⠀⢀⠀⠀⠀⠀⠀⠀⠠⠑⢀⠀⠀⠑⠌⠢⠀⢀⠀⠢⢡⠑⠑⢄⠀⠑⡄⡀⎥
⎢⠀⠀⠀⠀⠀⠑⢄⠀⠀⠑⢀⠠⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠂⠁⢄⠀⠠⡀⠀⠀⠑⠀⠀⠑⢄⠀⢕⢕⢐⠨⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠁⢄⠂⠀⠀⠑⢄⠀⠀⠀⠀⠀⠈⠀⡐⠄⠈⠢⡀⠀⠢⡀⠀⠀⠩⡐⡐⠕⢅⎦

In [31]:
function time_evolution(ψ::Vector{ComplexF64}, H::SparseMatrixCSC, dt::Float64)

    ψ_new = expmv(-im * dt, H, ψ)
    return normalize!(ψ_new)
end

time_evolution (generic function with 1 method)

In [32]:
function random_product_state(L::Int)
    ψ = nothing
    for i in 1:L
        θ1, θ2 = rand() * π, rand() * π
        ϕ1, ϕ2 = rand() * 2π, rand() * 2π
        c1 = cos(θ1 / 2)
        c2 = exp(im * ϕ1) * sin(θ1 / 2) * sin(θ2 / 2)
        c3 = exp(im * ϕ2) * sin(θ1 / 2) * cos(θ2 / 2)
        site_state = [c1, c2, c3]
        ψ = i == 1 ? site_state : kron(ψ, site_state)
    end
    return ψ / norm(ψ)
end

random_product_state (generic function with 1 method)

In [34]:
ψ = random_product_state(4)

81-element Vector{ComplexF64}:
    0.044816295941475945 + 0.0im
      0.0388565152469744 - 0.02841126152060851im
  -0.0008343410931777703 - 0.0027616272349132213im
    0.036979538392177755 - 0.11198759813706284im
    -0.03893255575279275 - 0.12053836751057613im
   -0.007589238296697294 - 0.00019385460408581032im
    -0.07762128311487154 - 0.245018028900266im
    -0.22262803423605754 - 0.16322697018864957im
   -0.013653199636285132 + 0.009344584391506436im
   0.0008083142016753585 + 0.004297696800231216im
                         ⋮
   -0.000306319221376958 + 6.64072969636416e-5im
 -0.00022348527928420564 + 0.00025176716230726557im
    9.794805766411598e-6 + 1.763941332011739e-5im
   -8.681551296696013e-5 + 0.0008202298802384667im
   0.0004447136228577284 + 0.0007661903414619676im
   5.2159663734869936e-5 - 9.920485407658448e-6im
   0.0008936007577455997 + 0.0015596807091959838im
   0.0017635261086474821 + 0.0007857729358077259im
    7.947307595106778e-5 - 8.410105785114708e-5im

In [35]:
norm(ψ)

1.0

In [36]:
ω = exp(2im * pi / 3)

-0.4999999999999998 + 0.8660254037844387im

In [ ]:
function Entropy_t_z2(L::Int, T::Float64, dt::Float64, p::Float64, shot::Int)
    
    # Initialize state
    s_t = random_product_state(L)
    
    
    # Build Hamiltonians once per shot
    H = Hamiltonian(L)

    ω = exp(2im * pi / 3)
    
    # Build a list of single-site Z operators for measurement
    Zl = [create_local_Z_operator(L, i) for i in 1:L]
    Z2l = [create_local_Z2_operator(L, i) for i in 1:L]
    
    # Initialize lists to store results
    S_list = Float64[]

    steps = Int(floor(T / dt))

    for _ in 1:steps
        
        # Record half-chain entropy
        push!(S_list, entropy_vn(s_t, L, 1:L÷2))

        # Time evolution
        s_t = time_evolution(s_t, H, dt)

        # Measurements
        if p != 0
            for l in 1:L
                if rand() < p ## edit this
                    p_m_zero  = 0.333 + 0.333 * real(s_t' * (Zl[l] + Z2l[l]) * s_t)
                    p_m_one = 0.333 + 0.333 * real(s_t' * (ω' * Zl[l] - (ω')^2 * Z2l[l]) * s_t)
                    x1 = rand()
                    if x1 < p_m_zero
                        s_t = 0.333 * ((Zl[l] - Z2l[l]) * s_t) / sqrt(p_m_zero)
                    elseif x1 < (p_m_one + p_m_zero)
                        s_t = 0.333 * ((Zl[l] + Z2l[l]) * s_t) / sqrt(p_m_one)
                    else
                        s_t = (s_t - Z2l[l] * s_t) / sqrt(1 - p_m_zero - p_m_one)
                    end
                end
            end
        end
    end

    
    
    base_folder = "/Users/uditvarma/Project_Data/z3-potts"

    today_date = Dates.format(Dates.today(), "yyyy-mm-dd")
    parent_dir = dirname(base_folder)
    folder_name = basename(base_folder) * "_" * today_date
    folder = joinpath(parent_dir, folder_name)
    mkpath(folder)    
    filename_entropy = joinpath(folder, "L$(L),T$(T),dt$(dt),p$(p),dirZ2,s$(shot)_hc.npy")
    npzwrite(filename_entropy, S_list)
    
    """
    filename_entropy = "L$(L),T$(T),dt$(dt),p$(p),dirZ2b,s$(shot)_hc.npy"
    npzwrite(filename_entropy, S_list)
    """

    return S_list
end

### Haar-random $Z_3$

In [41]:
using LinearAlgebra
using Random

# Generate Haar-random unitary (complex)
function haar_unitary(n::Int)
    A = randn(ComplexF64, n, n)
    Q, R = qr(A)
    phases = Diagonal(R) ./ abs.(Diagonal(R))
    return Q * Diagonal(phases)
end

# Original Z3 block-diagonal Haar unitary in basis:
# {00,12,21 | 01,10,22 | 02,20,11}
function dense_blockdiag(A::AbstractMatrix, B::AbstractMatrix, C::AbstractMatrix)
    m = size(A,1) + size(B,1) + size(C,1)
    M = zeros(eltype(A), m, m)
    M[1:3, 1:3] = A
    M[4:6, 4:6] = B
    M[7:9, 7:9] = C
    return M
end

function z3_haar_unitary()
    U0 = haar_unitary(3)
    U1 = haar_unitary(3)
    U2 = haar_unitary(3)
    return dense_blockdiag(U0, U1, U2)
end


# Permutation matrix to reorder from Z3 basis → standard lexicographic basis:
# Z3 basis index order:     1    2    3    4    5    6    7    8    9
#                           00,  12,  21,  01,  10,  22,  02,  20,  11
#
# Standard basis order:     00, 01, 02, 10, 11, 12, 20, 21, 22
#
# So: standard_index(i) = position of Z3_basis[i] in standard order
function permutation_matrix()
    # Map each Z3-basis vector to its standard basis index
    z3_basis = ["00","12","21","01","10","22","02","20","11"]
    std_basis = ["00","01","02","10","11","12","20","21","22"]

    P = zeros(Float64, 9, 9)
    for (i, state) in enumerate(z3_basis)
        j = findfirst(==(state), std_basis)
        P[j, i] = 1.0
    end
    return P
end

# Transform U_old into the new basis:
function transform_to_standard_basis(U_old)
    P = permutation_matrix()
    return P * U_old * P'
end

# Example
U_z3 = z3_haar_unitary()
U_standard = transform_to_standard_basis(U_z3)

9×9 Matrix{ComplexF64}:
 -0.117507+0.246003im        0.0+0.0im        …         0.0+0.0im
       0.0+0.0im       -0.328432-0.682508im      0.00377548-0.322326im
       0.0+0.0im             0.0+0.0im                  0.0+0.0im
       0.0+0.0im        0.606261-0.0613483im      -0.639018-0.126708im
       0.0+0.0im             0.0+0.0im                  0.0+0.0im
  0.663052+0.123162im        0.0+0.0im        …         0.0+0.0im
       0.0+0.0im             0.0+0.0im                  0.0+0.0im
 -0.622123+0.289536im        0.0+0.0im                  0.0+0.0im
       0.0+0.0im       0.0271649+0.232941im       0.0473542+0.685164im

In [64]:
function get_z3_operators()
    id = sparse(ComplexF64[1 0 0; 0 1 0; 0 0 1])
    τ = sparse(ComplexF64[1 0 0; 0 exp(2im * pi / 3) 0; 0 0 exp(4im * pi / 3)]) ## τ in Romain's paper

    return id, τ
end

function build_term(operators::Vector{<:SparseMatrixCSC}) #
    term = operators[1]
    for j in 2:length(operators)
        term = kron(term, operators[j])
    end
    return term
end

function create_local_tau_operator(L::Int, site::Int)
    id, τ = get_z3_operators()
    ops = fill(id, L)
    ops[site] = (im / sqrt(3)) * (τ' - τ)
    return build_term(ops)
end

create_local_tau_operator (generic function with 1 method)

In [96]:
using LinearAlgebra
using SparseArrays
using Random

# ---------------------------------------------------------
# Haar-random unitary (3×3), converted to sparse
# ---------------------------------------------------------
function haar_unitary_sparse(n::Int)
    A = randn(ComplexF64, n, n)
    Q, R = qr(A)
    phases = Diagonal(R) ./ abs.(Diagonal(R))
    return sparse(Q * Diagonal(phases))
end

# ---------------------------------------------------------
# Build 9×9 Z3-conserving Haar unitary as sparse blockdiag
#
# Basis ordering (Z3 sectors):
#   charge 0: 00, 12, 21
#   charge 1: 01, 10, 22
#   charge 2: 02, 20, 11
#
# So U_z3 is blockdiag(U0, U1, U2)
# ---------------------------------------------------------
function z3_haar()
    U0 = haar_unitary_sparse(3)
    U1 = haar_unitary_sparse(3)
    U2 = haar_unitary_sparse(3)
    return blockdiag(U0, U1, U2)  # built-in sparse blockdiag
end

# ---------------------------------------------------------
# Sparse permutation matrix: Z3 basis → standard lexicographic basis
#
# Z3 basis order:
#   ["00","12","21",  "01","10","22",  "02","20","11"]
#
# Standard basis order:
#   ["00","01","02",  "10","11","12",  "20","21","22"]
# ---------------------------------------------------------
function permutation_matrix_sparse()
    z3  = ["00","12","21","01","10","22","02","20","11"]
    std = ["00","01","02","10","11","12","20","21","22"]

    rows = Int[]
    cols = Int[]
    vals = ComplexF64[]

    for (i, state) in enumerate(z3)
        j = findfirst(==(state), std)
        push!(rows, j)
        push!(cols, i)
        push!(vals, 1.0 + 0im)
    end

    return sparse(rows, cols, vals, 9, 9)
end

# ---------------------------------------------------------
# Apply sparse basis change: U_std = P * U_z3 * P'
# ---------------------------------------------------------
function transform(U_z3)
    P = permutation_matrix_sparse()
    return P * U_z3 * P'
end

transform (generic function with 1 method)

In [73]:
x = create_local_tau_operator(2, 2)

9×9 SparseMatrixCSC{ComplexF64, Int64} with 6 stored entries:
     ⋅          ⋅           ⋅      …      ⋅          ⋅           ⋅    
     ⋅      1.0+0.0im       ⋅             ⋅          ⋅           ⋅    
     ⋅          ⋅      -1.0+0.0im         ⋅          ⋅           ⋅    
     ⋅          ⋅           ⋅             ⋅          ⋅           ⋅    
     ⋅          ⋅           ⋅             ⋅          ⋅           ⋅    
     ⋅          ⋅           ⋅      …      ⋅          ⋅           ⋅    
     ⋅          ⋅           ⋅             ⋅          ⋅           ⋅    
     ⋅          ⋅           ⋅             ⋅      1.0+0.0im       ⋅    
     ⋅          ⋅           ⋅             ⋅          ⋅      -1.0+0.0im

In [85]:
U_z3_sparse = z3_haar_unitary_sparse()
U_standard_sparse = transform_to_standard_basis_sparse(U_z3_sparse)

9×9 SparseMatrixCSC{ComplexF64, Int64} with 27 stored entries:
 -0.266944+0.255519im            ⋅            …            ⋅    
           ⋅           -0.044627-0.284971im       -0.38284-0.250582im
           ⋅                     ⋅                         ⋅    
           ⋅            0.397762+0.325871im      0.0840512-0.713109im
           ⋅                     ⋅                         ⋅    
  0.525099+0.460012im            ⋅            …            ⋅    
           ⋅                     ⋅                         ⋅    
  0.574347-0.215025im            ⋅                         ⋅    
           ⋅           -0.807278-0.0263777im     -0.120144-0.510509im

In [86]:
U_standard_sparse * x - x * U_standard_sparse

9×9 SparseMatrixCSC{ComplexF64, Int64} with 18 stored entries:
           ⋅                    ⋅            …             ⋅    
           ⋅                    ⋅                  0.76568+0.501165im
           ⋅                    ⋅                          ⋅    
           ⋅           0.397762+0.325871im      -0.0840512+0.713109im
           ⋅                    ⋅                          ⋅    
  0.525099+0.460012im           ⋅            …             ⋅    
           ⋅                    ⋅                          ⋅    
 -0.574347+0.215025im           ⋅                          ⋅    
           ⋅           -1.61456-0.0527554im                ⋅    

In [87]:
id = sparse(ComplexF64[1 0 0; 0 1 0; 0 0 1])
τ = sparse(ComplexF64[1 0 0; 0 exp(2im * pi / 3) 0; 0 0 exp(4im * pi / 3)])

3×3 SparseMatrixCSC{ComplexF64, Int64} with 3 stored entries:
 1.0+0.0im       ⋅                ⋅    
     ⋅      -0.5+0.866025im       ⋅    
     ⋅           ⋅           -0.5-0.866025im

In [92]:
((im / sqrt(3)) * (τ' - τ))

3×3 SparseMatrixCSC{ComplexF64, Int64} with 2 stored entries:
     ⋅          ⋅           ⋅    
     ⋅      1.0+0.0im       ⋅    
     ⋅          ⋅      -1.0+0.0im

In [ ]:
function time_evolution(ψ::Vector{ComplexF64}, L)

    U_odd  = odd_layer(L)
    U_even = even_layer(L)

    # One full brick-wall step:
    U_step = U_even * U_odd
    ψ_new = U_step * ψ

    return normalize!(ψ_new)
end

In [95]:
transform_to_standard_basis_sparse(z3_haar_unitary_sparse())

9×9 SparseMatrixCSC{ComplexF64, Int64} with 27 stored entries:
    -0.44676-0.339672im            ⋅            …             ⋅    
             ⋅           -0.117441-0.679162im       -0.117468-0.000581917im
             ⋅                     ⋅                          ⋅    
             ⋅           -0.517214-0.498678im      -0.0342511-0.0946855im
             ⋅                     ⋅                          ⋅    
 -0.00423406-0.119226im            ⋅            …             ⋅    
             ⋅                     ⋅                          ⋅    
   -0.150764-0.805025im            ⋅                          ⋅    
             ⋅           0.0758267-0.0548372im       -0.38569+0.909564im

In [107]:
transform(z3_haar())

9×9 SparseMatrixCSC{ComplexF64, Int64} with 27 stored entries:
 -0.0705684-0.383401im             ⋅           …            ⋅    
            ⋅            -0.820462-0.417519im     -0.136176+0.159403im
            ⋅                      ⋅                        ⋅    
            ⋅             0.189639+0.321081im     -0.604186+0.336779im
            ⋅                      ⋅                        ⋅    
   0.230508-0.232307im             ⋅           …            ⋅    
            ⋅                      ⋅                        ⋅    
   0.609495-0.607816im             ⋅                        ⋅    
            ⋅           -0.0937446+0.068387im      0.626277+0.29217im

In [97]:
function odd_layer(L::Int)
    U = transform(z3_haar())
    for i in 3:2:L-1
        U = kron(U, transform(z3_haar()))
    end
    if isodd(L)
        U = kron(U, id)   # pad last site if odd number of spins
    end
    return U
end

function even_layer(L::Int)
    U = id                          # start with a free spin at site 1
    for _ in 2:2:L-1
        U = kron(U, transform(z3_haar()))
    end
    if isodd(L)
        # if L is even, we’ve already used up all spins
        return U
    else
        # if L is odd, pad last site
        return kron(U, id)
    end
end

even_layer (generic function with 1 method)

In [101]:
odd_layer(4)

81×81 SparseMatrixCSC{ComplexF64, Int64} with 729 stored entries:
⎡⢑⢔⠌⠌⡂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢑⢔⠌⠌⡂⠀⠀⠀⠀⢑⢔⠌⠌⡂⠀⠀⠀⠀⎤
⎢⡂⠅⢕⢕⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡂⠅⢕⢕⠀⠀⠀⠀⠀⡂⠅⢕⢕⠀⠀⠀⠀⠀⎥
⎢⠈⠈⠀⠀⠑⢄⢄⢂⠢⠀⠀⠀⠀⠐⢄⢄⢂⠢⠀⠀⠀⠀⠈⠈⠀⠀⠁⠀⠀⠀⠀⠈⠈⠀⠀⠑⢄⢄⢂⠢⎥
⎢⠀⠀⠀⠀⠠⢑⢑⢔⠌⠀⠀⠀⠀⠠⢑⢑⢔⠌⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⢑⢑⢔⠌⎥
⎢⠀⠀⠀⠀⠈⠂⠂⠁⠑⢄⢀⠠⠠⡈⠂⠂⠁⠑⢄⢀⠄⢄⠀⠀⠀⠀⠠⡀⡀⠄⢄⠀⠀⠀⠀⠈⠂⠂⠁⠑⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⡐⠕⢅⢅⠂⠀⠀⠀⠀⡐⠽⡨⡐⠀⠀⠀⠀⢀⠪⠪⡨⡐⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⢀⠀⠀⡀⡀⠢⠡⠑⠑⢄⠀⠀⡀⡀⠢⠩⠊⠢⠀⠀⠀⠀⠐⠌⠌⠊⠢⠀⠀⠀⠀⢀⠀⠀⡀⡀⎥
⎢⠀⠀⠀⠀⠀⢕⢕⢐⠨⠀⠀⠀⠀⠀⢕⢕⢐⠨⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢕⢕⢐⠨⎥
⎢⠀⠀⠀⠀⠨⡐⡐⠕⢅⠀⠀⠀⠀⠨⡐⡐⠕⢅⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠨⡐⡐⠕⢅⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⢑⣔⡌⡌⡂⠀⠀⠀⠀⢑⣴⢡⢑⠀⠀⠀⠀⠈⣢⣢⢡⢑⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⢅⢂⠪⠪⡀⠀⠀⠀⠀⢅⢒⠕⢅⠀⠀⠀⠀⠨⡐⡐⠕⢅⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢑⢔⠌⠌⡂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢑⢔⠌⠌⡂⠀⠀⠀⠀⢑⢔⠌⠌⡂⠀⠀⠀⠀⎥
⎢⡂⠅⢕⢕⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡂⠅⢕⢕⠀⠀⠀⠀⠀⡂⠅⢕⢕⠀⠀⠀⠀⠀⎥
⎢⠈⠈⠀⠀⠁⠀⠀⠀⠀⠢⡠⡐⡐⠄⠀⠀⠀⠀⠢⣠⢂⠢⠈⠈⠀⠀⠑⢄⢄⢂⠢⠈⠈⠀⠀⠁⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠌⡊⡢⡢⠁⠀⠀⠀⠀⠌⣚⢔⠌⠀⠀⠀⠀⠠⢑⢑⢔⠌⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢄⢀⠠⠠⡀⠀⠀⠀⠀⠑⠐⠈⠈⠂⠀⠀⠀⠀⠑⠐⠁⠑⢄⢀⠠⠠⡈⠂⠂⠁⠑⢄⢀⠠⠠⡀⠀⠀⠀⠀⎥
⎢⡐⠕⢅⢅⠂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡐⠕⢅⢅⠂⠀⠀⠀⠀⡐⠕⢅⢅⠂⠀⠀⠀⠀⎥
⎢⠢⠡⠑⠑⢄⠀⠀⡀⡀⠀⠀⠀⠀⢀⠀⠀⡀⡀⠀⠀⠀⠀⠢⠡⠑⠑⠄⠀⠀⠀⠀⠢⠡⠑⠑⢄⠀⠀⡀⡀⎥
⎢⠀⠀⠀⠀⠀⢕⢕⢐⠨⠀⠀⠀⠀⠀⢕⢕⢐⠨⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢕⢕⢐⠨⎥
⎣⠀⠀⠀⠀⠨⡐⡐⠕⢅⠀⠀⠀⠀⠨⡐⡐⠕⢅⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠨⡐⡐⠕⢅⎦

In [103]:
even_layer(4) 

81×81 SparseMatrixCSC{ComplexF64, Int64} with 243 stored entries:
⎡⠑⢄⠀⠀⢀⠀⠀⠈⠢⠀⠈⠢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠀⠑⢄⠀⠑⢄⠀⠀⢄⠀⠀⠈⠂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠐⢄⠀⠑⢄⠀⠁⠀⠀⠁⠀⠢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⡀⠀⠀⠑⠄⠀⠑⢄⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠈⠂⠀⢄⠀⠀⢄⠀⠑⢄⠀⠑⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠢⡀⠀⠀⠁⠀⠀⠑⢄⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠈⠢⠀⠈⠢⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⢀⠀⠀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⢄⠀⠀⠑⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠱⡀⠀⠢⡀⠀⠀⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⡀⠑⢄⠀⠀⣀⠀⠀⠑⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠐⢄⠀⠀⠈⠀⠀⠑⢄⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⠀⠀⠈⠢⠀⠘⢄⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⢄⠀⠀⠑⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠁⠀⠀⠁⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠢⡀⠀⠢⡀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠑⢄⠀⠀⢀⠀⠀⠈⠢⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢄⠀⠑⢄⠀⠑⠀⠀⠑⠀⠠⡀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠑⢄⠀⠐⢄⠀⠀⠈⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠢⠀⢀⠀⠀⢀⠀⠑⢄⠀⠑⠄⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⡀⠀⠀⠑⠀⠀⠑⢄⠀⠑⢄⠀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠢⡀⠀⠢⡀⠀⠀⠁⠀⠀⠑⢄⎦

In [110]:
L = 6

U_odd  = odd_layer(L)
U_even = even_layer(L)

# One full brick-wall step:
U_step = U_even * U_odd

729×729 SparseMatrixCSC{ComplexF64, Int64} with 177147 stored entries:
⎡⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎤
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎣⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎦